In [1]:
import pandas as pd
from sklearn.metrics import classification_report


In [2]:
df = pd.read_csv('german_credit_data (1).csv')

In [3]:
df.isna().sum()

Unnamed: 0            0
Age                   0
Sex                   0
Job                   0
Housing               0
Saving accounts     183
Checking account    394
Credit amount         0
Duration              0
Purpose               0
Risk                  0
dtype: int64

In [4]:
df.head()

,Unnamed: 0,Age,Sex,Job,Housing,Saving accounts,Checking account,Credit amount,Duration,Purpose,Risk
0,0,67,male,2,own,NaN,little,1169,6,radio/TV,good
1,1,22,female,2,own,little,moderate,5951,48,radio/TV,bad
2,2,49,male,1,own,little,NaN,2096,12,education,good
3,3,45,male,2,free,little,little,7882,42,furniture/equipment,good
4,4,53,male,2,free,little,little,4870,24,car,bad


In [5]:
#df[['Saving accounts','Checking account']]

In [6]:
#for each in range(0,len(df)):
    #print(type(df.loc[each]['Checking account']))
    #if type(df.loc[each]['Checking account'])==float and type(df.loc[each]['Saving accounts'])!=float :
        #df.at[each,'Checking account'] = df.loc[each]['Saving accounts']
        #print(df.loc[each]['Checking account'])

In [7]:
df.isna()['Checking account'].sum()

394

In [8]:
df.isna()['Saving accounts'][1]

False

In [9]:
for each in range(0,len(df)):
    if df.isna()['Checking account'][each] == True and df.isna()['Saving accounts'].isna()[each]==False:
        df.at[each,'Checking account'] = df.loc[each]['Saving accounts']
    elif df.isna()['Saving accounts'][each] == True and df.isna()['Checking account'].isna()[each]==False:
        df.at[each,'Saving accounts'] = df.loc[each]['Checking account']
    


In [10]:
df.isna().sum()

Unnamed: 0           0
Age                  0
Sex                  0
Job                  0
Housing              0
Saving accounts     99
Checking account    99
Credit amount        0
Duration             0
Purpose              0
Risk                 0
dtype: int64

In [11]:
df = df.dropna()

In [12]:
df.isna().sum()

Unnamed: 0          0
Age                 0
Sex                 0
Job                 0
Housing             0
Saving accounts     0
Checking account    0
Credit amount       0
Duration            0
Purpose             0
Risk                0
dtype: int64

In [13]:
df.head()

,Unnamed: 0,Age,Sex,Job,Housing,Saving accounts,Checking account,Credit amount,Duration,Purpose,Risk
0,0,67,male,2,own,little,little,1169,6,radio/TV,good
1,1,22,female,2,own,little,moderate,5951,48,radio/TV,bad
2,2,49,male,1,own,little,little,2096,12,education,good
3,3,45,male,2,free,little,little,7882,42,furniture/equipment,good
4,4,53,male,2,free,little,little,4870,24,car,bad


In [14]:
from sklearn.preprocessing import LabelEncoder

In [15]:
##Convert categorical to numerical
for col in df.columns:
    if df[col].dtype == object:
        le = LabelEncoder()
        le.fit(df[col])
        df[col]=(le.transform(df[col]))

In [16]:
df.head()

,Unnamed: 0,Age,Sex,Job,Housing,Saving accounts,Checking account,Credit amount,Duration,Purpose,Risk
0,0,67,1,2,1,0,0,1169,6,5,1
1,1,22,0,2,1,0,1,5951,48,5,0
2,2,49,1,1,1,0,0,2096,12,3,1
3,3,45,1,2,0,0,0,7882,42,4,1
4,4,53,1,2,0,0,0,4870,24,1,0


In [17]:
X = df.drop(columns='Risk')
y = df['Risk']

In [18]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [19]:
import seaborn as sns
import matplotlib.pyplot as plt
plt.figure(figsize = (16,5))

sns.heatmap(df.corr(),annot=True)


In [20]:
from sklearn.preprocessing import StandardScaler

sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [21]:
from sklearn.decomposition import PCA
pca = PCA()
X_train_PCA = pca.fit_transform(X_train)
X_test_PCA = pca.fit_transform(X_test)

In [22]:
pca.explained_variance_ratio_

array([0.27475638, 0.13110885, 0.11776919, 0.09699175, 0.08464982,
       0.08059802, 0.0682865 , 0.054192  , 0.04829755, 0.04334994])

The first 4 components are responsible for 65% of the variance aka 65% of the classification information we need.

In [23]:
from sklearn.decomposition import PCA
pca = PCA(n_components = 4)
X_train_PCA = pca.fit_transform(X_train)
X_test_PCA = pca.fit_transform(X_test)

In [24]:
from sklearn.ensemble import RandomForestClassifier

classifier = RandomForestClassifier(max_depth=2, random_state=0)
classifier.fit(X_train_PCA, y_train)

# Predicting the Test set results
y_pred = classifier.predict(X_test_PCA)

//anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


In [25]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

cm = confusion_matrix(y_test, y_pred)
print(cm)
print('Accuracy ' + str(accuracy_score(y_test, y_pred)))

[[ 17  40]
 [ 11 113]]
Accuracy 0.7182320441988951


Predicting without PCA

In [26]:
classifier = RandomForestClassifier(max_depth=2, random_state=0)
classifier.fit(X_train, y_train)

# Predicting the Test set results
y_pred = classifier.predict(X_test)
cm = confusion_matrix(y_test, y_pred)
print(cm)
#print('Accuracy ' + str(accuracy_score(y_test, y_pred)))
print(classification_report(y_test, y_pred))

[[  7  50]
 [  4 120]]
              precision    recall  f1-score   support

           0       0.64      0.12      0.21        57
           1       0.71      0.97      0.82       124

    accuracy                           0.70       181
   macro avg       0.67      0.55      0.51       181
weighted avg       0.68      0.70      0.62       181



//anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


We observe that our results are better after PCA

In [27]:
X_train_PCA.shape

(720, 4)

In [28]:
from sklearn.linear_model import LogisticRegression
logistic_reg = LogisticRegression()

In [29]:
logistic_reg.fit(X_train_PCA,y_train)

//anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [30]:
y_pred = logistic_reg.predict(X_test_PCA)
cm = confusion_matrix(y_test,y_pred)
print(cm)
#print('Accuracy '+str(accuracy_score(y_test,y_pred)))

[[ 14  43]
 [ 10 114]]


In [45]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.58      0.25      0.35        57
           1       0.73      0.92      0.81       124

    accuracy                           0.71       181
   macro avg       0.65      0.58      0.58       181
weighted avg       0.68      0.71      0.66       181



Since cost of a false negative is high in this case, we will use recall as a metric.